# Implementing an image super resolution.
## Here I'm going to do some quick training as proof of concept (PoC).

### Imports

In [1]:
import gc
from models import generator_no_residual, generator_with_residual, discriminator
from utils import check_path_exists
from data_loader import load_images_with_truth
from loss_functions import perceptual_loss, perceptual_loss_16, perceptual_loss_19
from loss_functions import texture_loss_multi_layers, perceptual_plus_texture_loss, perceptual_16_plus_texture_loss
from visualizations import plot_images_for_compare, plot_images_for_compare_separate, compare_models, compare_models_single_image
from keras.callbacks import ModelCheckpoint, EarlyStopping
from keras.optimizers import Adam
%matplotlib inline

Using TensorFlow backend.


### Variables declaration

In [2]:
# Enable this flag if running on the laptop with smaller GPU.
running_on_laptop = False

# This will use the smaller datasets (train_small, val_small, test_small).
use_small_dataset = True

# Set to true if you want to use CelebA dataset. Otherwise it will use MS COCO.
use_dataset_celeba = False

# Set to true and it will not execute training. Usefull when just want to plot the results.
disable_training = False

# What to be the verbose level during training.
training_verbose = 2

enable_p = True
enable_p16 = True
enable_p19 = True
enable_t = True
enable_pt = True
enable_pt16 = True
enable_pt16_bci = True
enable_pt_bci = True
enable_pt16_no_res = True

train_epochs = 100
train_batch_size = 32
test_image_index_to_show = range(20)
optimizer = Adam(lr=0.0001)

if running_on_laptop:
    train_epochs = 100
    train_batch_size = 8
    test_image_index_to_show = range(20)
    optimizer = Adam(lr=0.001)

In [3]:
# Define the dataset path
dataset = "MSCOCO"
if use_dataset_celeba:
    dataset = "celeba"

if not use_small_dataset:
    train_dataset_path = './data/{0}/train/*'.format(dataset)
    validation_dataset_path = './data/{0}/val/*'.format(dataset)
    test_dataset_path = './data/{0}/test/*'.format(dataset)
else:
    train_dataset_path = './data/{0}/train_small/*'.format(dataset)
    validation_dataset_path = './data/{0}/val_small/*'.format(dataset)
    test_dataset_path = './data/{0}/test_small/*'.format(dataset)

print(train_dataset_path)
print(validation_dataset_path)
print(test_dataset_path)

./data/MSCOCO/train/*
./data/MSCOCO/val/*
./data/MSCOCO/test/*


In [4]:
perceptual_loss_checkpint_path = './saved_models/weights.best.train.mscoco.pl.hdf5'
perceptual_loss_16_checkpoint_path = './saved_models/weights.best.train.mscoco.pl16.hdf5'
perceptual_loss_19_checkpoint_path = './saved_models/weights.best.train.mscoco.pl19.hdf5'
texture_loss_ml_checkpoint_path = './saved_models/weights.best.train.mscoco.tl_ml.hdf5'
texture_plus_perceptual_loss_checkpoint_path = './saved_models/weights.best.train.mscoco.tl_plus_pl.hdf5'
perceptual_16_plus_texture_loss_checkpoint_path = './saved_models/weights.best.train.mscoco.pl16_plus_tl.hdf5'
perceptual_16_plus_texture_loss_bci_checkpoint_path = './saved_models/weights.best.train.mscoco.pl16_plus_tl_bci.hdf5'
perceptual_plus_texture_loss_bci_checkpoint_path = './saved_models/weights.best.train.mscoco.pl_plus_tl_bci.hdf5'
perceptual_16_plus_texture_loss_no_res_checkpoint_path = './saved_models/weights.best.train.mscoco.pl16_plus_tl_no_res.hdf5'

if use_dataset_celeba:
    perceptual_loss_checkpint_path = './saved_models/weights.best.train.celeba.pl.hdf5'
    perceptual_loss_16_checkpoint_path = './saved_models/weights.best.train.celeba.pl16.hdf5'
    perceptual_loss_19_checkpoint_path = './saved_models/weights.best.train.celeba.pl19.hdf5'
    texture_loss_ml_checkpoint_path = './saved_models/weights.best.train.celeba.tl_ml.hdf5'
    texture_plus_perceptual_loss_checkpoint_path = './saved_models/weights.best.train.celeba.tl_plus_pl.hdf5'
    perceptual_16_plus_texture_loss_checkpoint_path = './saved_models/weights.best.train.celeba.pl16_plus_tl.hdf5'
    perceptual_16_plus_texture_loss_bci_checkpoint_path = './saved_models/weights.best.train.celeba.pl16_plus_tl_bci.hdf5'
    perceptual_plus_texture_loss_bci_checkpoint_path = './saved_models/weights.best.train.celeba.pl_plus_tl_bci.hdf5'
    perceptual_16_plus_texture_loss_no_res_checkpoint_path = './saved_models/weights.best.train.celeba.pl16_plus_tl_no_res.hdf5'

### Helper functions

#### Train the model

In [ ]:
# NOTE: Some of the parameters are used from the global space
def model_train(model, optimizer, loss_function, checkpoint_path, verbose=2):
    model.compile(optimizer=optimizer, loss=loss_function, metrics=['accuracy'])
    checkpointer = ModelCheckpoint(filepath=checkpoint_path, 
                               verbose=verbose, save_best_only=True)
    early_stopper = EarlyStopping(monitor='val_loss', min_delta=0, patience=20, verbose=verbose)

    model.fit(train_data_tensors, train_truth_tensors,
              validation_data=(validation_data_tensors, validation_truth_tensors),
              epochs=train_epochs, batch_size=train_batch_size, callbacks=[checkpointer, early_stopper], verbose=verbose)

#### Predict with the model

In [ ]:
# NOTE: Some of the parameters are used from the global space
def model_predict(model, checkpoint_path):
    return
    model.load_weights(checkpoint_path)

    print('Predicting...')
    predictions = model.predict(test_data_tensors)

    print('Plotting the results...')
    plot_images_for_compare_separate(test_data_tensors, predictions, test_truth_tensors, test_image_index_to_show)

    print('All done!')

In [ ]:
# NOTE: Some of the parameters are used from the global space
def model_predict_2(model, checkpoint_path):
    model.load_weights(checkpoint_path)

    print('Predicting...')
    predictions = model.predict(test_data_tensors)

    print('Plotting the results...')
    plot_images_for_compare(test_data_tensors, predictions, test_truth_tensors, test_image_index_to_show)

    print('All done!')

### Function to create models, train and predict

In [ ]:
def model_p(get_model_only=False, summary=False):
    """
    """
    if not get_model_only:
        model = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
        if not disable_training:
            model_train(model=model, optimizer=optimizer, loss_function=perceptual_loss, checkpoint_path=perceptual_loss_checkpint_path,
                        verbose=training_verbose)
        model_predict(model, perceptual_loss_checkpint_path)
        # Free the memory
        del model
        gc.collect()
    
    # Recreate the model and return it
    return generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)

#### Define the functions to load the images

For training, we use all color images in MSCOCO [31]
that have at least 384 pixels on the short side resulting in
roughly 200k images. All images are cropped centrally to a
square and then downsampled to 256×256 to reduce noise
and JPEG artifacts. During training, we fix the size of the
input ILR to 32×32. As the scale of objects in the MSCOCO
dataset is too small when downsampled to such a small size,
we downsample the 256×256 images by  and then crop
these to patches of size 32×32. After training the model
for any given scaling factor , the input to the fully convolutional
network at test time can be an image of arbitrary
dimensions w×h which is then upscaled to (w)×(h).

#### Read the input images

In [ ]:
print('Loading train data:')
train_data, train_truth = load_images_with_truth(train_dataset_path, 4)
print('Loading validation data:')
validation_data, validation_truth = load_images_with_truth(validation_dataset_path, 4)
print('Loading test data:')
test_data, test_truth = load_images_with_truth(test_dataset_path, 4)

print("Train images: ", len(train_data))
print('Validation images: ', len(validation_data))
print("Test images: ", len(test_data))

train_data_tensors = train_data.astype('float32')/255
train_truth_tensors = train_truth.astype('float32')/255

validation_data_tensors = validation_data.astype('float32')/255
validation_truth_tensors = validation_truth.astype('float32')/255

test_data_tensors = test_data.astype('float32')/255
test_truth_tensors = test_truth.astype('float32')/255

Loading train data:


100%|██████████████████████████████████████████████████████████████████████████| 154168/154168 [34:40<00:00, 74.11it/s]


Loading validation data:


100%|████████████████████████████████████████████████████████████████████████████| 18056/18056 [04:05<00:00, 74.06it/s]


Loading test data:


100%|██████████████████████████████████████████████████████████████████████████████████| 20/20 [00:00<00:00, 68.08it/s]


Train images:  154168
Validation images:  18056
Test images:  20


### We need to create separate models even though that they are based on the same architecture. This is because we want to have new default parameters each time.

### Perceptual loss based on VGG19 (selected layers)

In [ ]:
if enable_p:
    model = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model, optimizer=optimizer, loss_function=perceptual_loss, checkpoint_path=perceptual_loss_checkpint_path, verbose=training_verbose)
    model_predict(model, perceptual_loss_checkpint_path)
    del model
    gc.collect()

Train on 154168 samples, validate on 18056 samples
Epoch 1/100
 - 1167s - loss: 0.1217 - acc: 0.7623 - val_loss: 0.1049 - val_acc: 0.7677

Epoch 00001: val_loss improved from inf to 0.10491, saving model to ./saved_models/weights.best.train.mscoco.pl.hdf5
Epoch 2/100
 - 1126s - loss: 0.1012 - acc: 0.7789 - val_loss: 0.0981 - val_acc: 0.7843

Epoch 00002: val_loss improved from 0.10491 to 0.09813, saving model to ./saved_models/weights.best.train.mscoco.pl.hdf5
Epoch 3/100
 - 1091s - loss: 0.0960 - acc: 0.7806 - val_loss: 0.0939 - val_acc: 0.7807

Epoch 00003: val_loss improved from 0.09813 to 0.09392, saving model to ./saved_models/weights.best.train.mscoco.pl.hdf5
Epoch 4/100
 - 1095s - loss: 0.0928 - acc: 0.7787 - val_loss: 0.0918 - val_acc: 0.7812

Epoch 00004: val_loss improved from 0.09392 to 0.09175, saving model to ./saved_models/weights.best.train.mscoco.pl.hdf5
Epoch 5/100
 - 1075s - loss: 0.0906 - acc: 0.7748 - val_loss: 0.0898 - val_acc: 0.7856

Epoch 00005: val_loss improve

### Peceptual loss based on VGG19 (block3_pool)

In [ ]:
if enable_p19:
    model19 = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model19, optimizer=optimizer, loss_function=perceptual_loss_19, checkpoint_path=perceptual_loss_19_checkpoint_path)
    model_predict(model=model19, checkpoint_path=perceptual_loss_19_checkpoint_path)
    del model19
    gc.collect()

### Peceptual loss based on VGG16 (block3_pool)

In [ ]:
if enable_p16:
    model16 = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model16, optimizer=optimizer, loss_function=perceptual_loss_16, checkpoint_path=perceptual_loss_16_checkpoint_path, verbose=training_verbose)
    model_predict(model=model16, checkpoint_path=perceptual_loss_16_checkpoint_path)
    del model16
    gc.collect()

### Texture loss (multi layers)

In [ ]:
if enable_t:
    model_tl_ml = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model_tl_ml, optimizer=optimizer, loss_function=texture_loss_multi_layers, checkpoint_path=texture_loss_ml_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_tl_ml, checkpoint_path=texture_loss_ml_checkpoint_path)
    del model_tl_ml
    gc.collect()

### Texture loss plus perceptual loss

In [ ]:
if enable_pt:
    model_tl_plus_pl = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model_tl_plus_pl, optimizer=optimizer, loss_function=perceptual_plus_texture_loss,
                    checkpoint_path=texture_plus_perceptual_loss_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_tl_plus_pl, checkpoint_path=texture_plus_perceptual_loss_checkpoint_path)
    del model_tl_plus_pl
    gc.collect()

### Peceptual loss 16 + texture loss

In [ ]:
if enable_pt16:
    model_pl_16_plus_tl = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    if not disable_training:
        model_train(model=model_pl_16_plus_tl, optimizer=optimizer, loss_function=perceptual_16_plus_texture_loss,
                    checkpoint_path=perceptual_16_plus_texture_loss_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_pl_16_plus_tl, checkpoint_path=perceptual_16_plus_texture_loss_checkpoint_path)
    del model_pl_16_plus_tl
    gc.collect()

## Let's try some with bicubic interpolation included

### Peceptual loss 16 + texture loss. Bicubic interpolation included in the model.

In [ ]:
if enable_pt16_bci:
    model_pl_16_plus_tl_bci = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=True)
    if not disable_training:
        model_train(model=model_pl_16_plus_tl_bci, optimizer=optimizer, loss_function=perceptual_16_plus_texture_loss,
                    checkpoint_path=perceptual_16_plus_texture_loss_bci_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_pl_16_plus_tl_bci, checkpoint_path=perceptual_16_plus_texture_loss_bci_checkpoint_path)
    del model_pl_16_plus_tl_bci
    gc.collect()

### Perceptual plus texture loss. Bicubic interpolation included in the model.

In [ ]:
if enable_pt_bci:
    model_pl_plus_tl_bci = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=True)
    if not disable_training:
        model_train(model=model_pl_plus_tl_bci, optimizer=optimizer, loss_function=perceptual_plus_texture_loss,
                    checkpoint_path=perceptual_plus_texture_loss_bci_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_pl_plus_tl_bci, checkpoint_path=perceptual_plus_texture_loss_bci_checkpoint_path)
    del model_pl_plus_tl_bci
    gc.collect()

### Peceptual loss 16 + texture loss. No residual network.

In [ ]:
if enable_pt16_no_res:
    model_pl_16_plus_tl_no_res = generator_no_residual(input_shape=train_data.shape[1:], summary=False)
    if not disable_training:
        model_train(model=model_pl_16_plus_tl_no_res, optimizer=optimizer, loss_function=perceptual_16_plus_texture_loss,
                    checkpoint_path=perceptual_16_plus_texture_loss_no_res_checkpoint_path, verbose=training_verbose)
    model_predict(model=model_pl_16_plus_tl_no_res, checkpoint_path=perceptual_16_plus_texture_loss_no_res_checkpoint_path)
    del model_pl_16_plus_tl_no_res
    gc.collect()

## Compare all of the models at the same time

In [ ]:
models_data = []

if enable_p:
    model = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "P", 'model': model, 'checkpoint': perceptual_loss_checkpint_path})

if enable_p16:
    model19 = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "P VGG19", 'model': model19, 'checkpoint': perceptual_loss_19_checkpoint_path})

if enable_p19:
    model16 = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "P VGG16", 'model': model16, 'checkpoint': perceptual_loss_16_checkpoint_path})

if enable_t:
    model_tl_ml = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "T", 'model': model_tl_ml, 'checkpoint': texture_loss_ml_checkpoint_path})

if enable_pt:
    model_tl_plus_pl = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "PT VGG19", 'model': model_tl_plus_pl, 'checkpoint': texture_plus_perceptual_loss_checkpoint_path})

if enable_pt16:
    model_pl_16_plus_tl = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=False)
    models_data.append({'name': "PT VGG16", 'model': model_pl_16_plus_tl, 'checkpoint': perceptual_16_plus_texture_loss_checkpoint_path})

if enable_pt16_bci:
    model_pl_16_plus_tl_bci = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=True)
    models_data.append({'name': "PT VGG16 (BCI)", 'model': model_pl_16_plus_tl_bci, 'checkpoint': perceptual_16_plus_texture_loss_bci_checkpoint_path})

if enable_pt_bci:
    model_pl_plus_tl_bci = generator_with_residual(input_shape=train_data.shape[1:], summary=False, add_bicubic=True)
    models_data.append({'name': "PT VGG19 (BCI)", 'model': model_pl_plus_tl_bci, 'checkpoint': perceptual_plus_texture_loss_bci_checkpoint_path})

if enable_pt16_no_res:
    model_pl_16_plus_tl_no_res = generator_no_residual(input_shape=train_data.shape[1:], summary=False)
    models_data.append({'name': "PT VGG16 (NR)", 'model': model_pl_16_plus_tl_no_res, 'checkpoint': perceptual_16_plus_texture_loss_no_res_checkpoint_path})

In [ ]:
# compare_models(test_data_tensors, test_truth_tensors, models_data, test_image_index_to_show, show_input=True)

## Plot the images in higher dimension

In [ ]:
compare_models_single_image(test_data_tensors, test_truth_tensors, models_data, test_image_index_to_show, show_input=True)